In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pliki-do-titanica/train.csv
/kaggle/input/pliki-do-titanica/test.csv


**Titanic - Machine Learning from Disaster**

In [2]:
train_data = pd.read_csv("/kaggle/input/pliki-do-titanica/train.csv")
print(train_data.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [3]:
test_data = pd.read_csv("/kaggle/input/pliki-do-titanica/test.csv")
print(test_data.head())

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


In [4]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of women who survived: 0.7420382165605095
% of men who survived: 0.18890814558058924


In [5]:
#sprawdzenie z ktorych informacji nie warto korzystac, a ktore mozna uzupelnic 
missing_val_count_by_column = (train_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

missing_val_count_by_column = (test_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Age         177
Cabin       687
Embarked      2
dtype: int64
Age       86
Fare       1
Cabin    327
dtype: int64


In [10]:
from sklearn.ensemble import RandomForestClassifier
#chcemy przewidziec kto przezyje
y = train_data["Survived"]

#zamieniamy wszystkie NaN na wartosc srednia wieku czyli 25
train_data['Age'] = train_data['Age'].replace(np.nan, 25)
test_data['Age'] = train_data['Age'].replace(np.nan, 25)

#wypelnienie wszystkich nieznanych pol w kolumnie koszt podruzy
train_data['Fare'] = train_data['Fare'].replace(np.nan, 0)
test_data['Fare'] = train_data['Fare'].replace(np.nan, 0)
#print(train_data.head())
#train_data.info()

#zmieniamy typ danych wieku na float32 aby drzewo moglo go obsluzyc

test_data['Age'] = test_data['Age'].astype('int32')
train_data['Age'] = train_data['Age'].astype('int32')
#zmieniamy typ danych wieku na float32 aby drzewo moglo go obsluzyc
test_data['Fare'] = test_data['Fare'].astype('int64')
train_data['Fare'] = train_data['Fare'].astype('int64')
#test_data.info()
#do przewidywan uzywamy klasy przedzialu, plci,ilosc rodzicow dzieci na pokladzie wieku oraz kosztu za bilet
features = ["Pclass", "Sex", "Parch","Age","Fare"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier( criterion="entropy",random_state=3)
model.fit(X, y)
predictions = model.predict(X_test)

In [11]:
#zapis wygenerowanych przewidywan do pliku
output = pd.DataFrame({'PassengerId': test_data.PassengerId,'Sex': test_data.Sex, 'Survived': predictions})
output.to_csv('my_submision.csv', index=False)
print("Your submission was successfully saved.")

ValueError: could not broadcast input array from shape (6) into shape (418)

In [ ]:
#sprawdzenie czy dobrze sie zapisal plik
test_my_submission = pd.read_csv("/kaggle/working/my_submision.csv")
print(test_my_submission.head())


#%kobiet przy zyciu
women = test_my_submission.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

#%mezczyzn przy zyciu
men = test_my_submission.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)
